<a href="https://colab.research.google.com/github/Preethikuppuri/Preethikuppuri/blob/main/data%20cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

# Create a dirty dataset with intentional issues
data = {
    'Name': [' alice ', 'BOB', 'Charly', 'Dinesh Kumar', '###', 'Alice', 'Bob'],
    'Age': [25, -30, 35, None, -999, 25, 30],
    'Gender': ['Male', 'M', 'F', 'female', 'male', 'Male', 'Male'],
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01'],
    'Sales_Q1': [100, 200, 300, 100, 400, 100, 200],
    'Sales_Q2': [150, 250, 350, 150, 450, 150, 250],
    'Weight': ['10kg', '15000g', '20kg', '20kg', '15kg', '10kg', '10kg'],
    'Text': ['Hello!', 'Data@Science', 'Python#3', b'This is a test.', 'Clean text.', 'ANOTHER!', 'Hello!'],
    'Values': [10, 12, 15, 1000, 14, 0, 10],
    'Unnecessary': [1, 2, 3, 4, 5, 6, 7],
    'Feature1': [1, 2, 3, 4, 5, 1, 2],
    'Feature2': [2, 4, 6, 8, 10, 2, 4],
    'Feature3': [3, 6, 9, 12, 15, 3, 6],
    'Sparse1': [0, 0, 0, 0, 0, 0, 0],
    'Sparse2': [0, 0, 0, 1, 0, 0, 0],
    'Class': ['A', 'A', 'A', 'B', 'B', 'A', 'A'],
    'Details': [{'Age': 25, 'City': 'Chennai'}, {'Age': 30, 'City': 'Mumbai'},
                {'Age': 35, 'City': 'Delhi'}, {'Age': 40, 'City': 'Hyderabad'},
                {'Age': 45, 'City': 'Kolkata'}, {'Age': 25, 'City': 'Chennai'}, {'Age': 30, 'City': 'Mumbai'}],
    'Future_Info': [150, 200, 250, 300, 350, 400, 450]
}

df_dirty = pd.DataFrame(data)
df_dirty.to_csv('dirty_dataset.csv', index=False)
df_dirty.head()


,Name,Age,Gender,Date,Sales_Q1,Sales_Q2,Weight,Text,Values,Unnecessary,Feature1,Feature2,Feature3,Sparse1,Sparse2,Class,Details,Future_Info
0,alice,25.0,Male,2023-01-01,100,150,10kg,Hello!,10,1,1,2,3,0,0,A,"{'Age': 25, 'City': 'Chennai'}",150
1,BOB,-30.0,M,2023-01-02,200,250,15000g,Data@Science,12,2,2,4,6,0,0,A,"{'Age': 30, 'City': 'Mumbai'}",200
2,Charly,35.0,F,2023-01-03,300,350,20kg,Python#3,15,3,3,6,9,0,0,A,"{'Age': 35, 'City': 'Delhi'}",250
3,Dinesh Kumar,NaN,female,2023-01-01,100,150,20kg,b'This is a test.',1000,4,4,8,12,0,1,B,"{'Age': 40, 'City': 'Hyderabad'}",300
4,###,-999.0,male,2023-01-01,400,450,15kg,Clean text.,14,5,5,10,15,0,0,B,"{'Age': 45, 'City': 'Kolkata'}",350


In [15]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from scipy.stats import zscore, ks_2samp
import string

# Load the dirty dataset
df = pd.read_csv('dirty_dataset.csv')

# STEP 1: Remove Duplicate Data
df = df.drop_duplicates()

# STEP 2: Handle Missing Values
df['Age'] = df['Age'].fillna(df['Age'].mean())

# STEP 3: Correct Data Types
df['Date'] = pd.to_datetime(df['Date'])

# STEP 4: Standardize Data
df['Name'] = df['Name'].str.title()

# STEP 5: Filter Outliers (on Age using IQR)
Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['Age'] >= Q1 - 1.5 * IQR) & (df['Age'] <= Q3 + 1.5 * IQR)]

# STEP 6: Normalize Data (Age)
df['Age'] = (df['Age'] - df['Age'].min()) / (df['Age'].max() - df['Age'].min())

# STEP 7: Remove Irrelevant Data
df = df.drop(columns=['Unnecessary'])

# STEP 8: Validate Data Integrity (print missing)
print(df.isnull().sum())

# STEP 9: Document Cleaning Steps (example)
cleaning_steps = ["Removed duplicates", "Handled missing values", "Standardized Name", "Normalized Age"]

# STEP 10: Remove Extra Spaces
df['Name'] = df['Name'].str.strip()

# STEP 11: Handle Inconsistent Categorical Data
df['Gender'] = df['Gender'].replace({'M': 'Male', 'F': 'Female', 'female': 'Female', 'male': 'Male'})

# STEP 12: Split Columns
df[['FirstName', 'LastName']] = df['Name'].str.split(n=1, expand=True).fillna('')

# STEP 13: Merge Columns
df['FullDate'] = df['Date'].dt.strftime('%Y-%m-%d')

# STEP 14: Remove Special Characters
df['Name'] = df['Name'].str.replace(r'[^\w\s]', '', regex=True)

# STEP 15: Convert to Lowercase
df['Gender'] = df['Gender'].str.lower()

# STEP 16: Remove Unwanted Rows
df = df[df['Age'] >= 0]

# STEP 17: Encode Categorical Variables
df = pd.get_dummies(df, columns=['Gender'], drop_first=True)

# STEP 18: Check for Data Consistency
print("Negative Ages:", df[df['Age'] < 0])

# STEP 19: Aggregate Data (by Date)
agg = df.groupby('Date')[['Sales_Q1', 'Sales_Q2']].sum().reset_index()

# STEP 20: Handle Mixed Data Types
df['Values'] = pd.to_numeric(df['Values'], errors='coerce')

# STEP 21: Detect and Remove Corrupted Data
df = df[~df['Name'].str.contains(r'[^a-zA-Z\s]', na=False)]
df = df[df['Age'] != -999]

# STEP 22: Reshape Data (wide to long)
df_melted = pd.melt(df, id_vars=['Name'], value_vars=['Sales_Q1', 'Sales_Q2'],
                    var_name='Quarter', value_name='Sales')

# STEP 23: Check for Skewness
print("Skewness of Values:", df['Values'].skew())
df['Values'] = df['Values'].apply(lambda x: np.log(x + 1))

# STEP 24: Handle Time Zones
df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize('UTC')

# STEP 25: Handle Imbalanced Data
df_major = df[df['Class'] == 'A']
df_minor = df[df['Class'] == 'B']
df_minor_upsampled = resample(df_minor, replace=True, n_samples=len(df_major), random_state=42)
df = pd.concat([df_major, df_minor_upsampled])

# STEP 26: Remove Multicollinearity
corr_matrix = df.corr(numeric_only=True)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
drop_cols = [col for col in upper.columns if any(upper[col] > 0.95)]
df = df.drop(columns=drop_cols)

# STEP 27: Handle Noisy Data (moving average)
df['Smoothed'] = df['Values'].rolling(window=2).mean()

# STEP 28: Handle Data Leakage
df = df.drop(columns=['Future_Info'])

# STEP 29: Flatten Hierarchical Data
# First: convert strings to dictionaries safely
import ast

df['Details'] = df['Details'].apply(lambda x: ast.literal_eval(str(x)))

# Normalize the nested dictionaries
details_df = pd.json_normalize(df['Details'])

# Optional: rename to avoid column name conflicts
details_df.columns = ['Details_' + col for col in details_df.columns]

# Reset index to prevent duplicate index issues
df = df.reset_index(drop=True)
details_df = details_df.reset_index(drop=True)

# Now safe to concat
df = pd.concat([df.drop(columns=['Details']), details_df], axis=1)

# STEP 30: Detect Data Drift (KS test)
train = df[df['Date'] < '2023-01-02']['Values']
test = df[df['Date'] >= '2023-01-02']['Values']
stat, p = ks_2samp(train, test)
print("P-value for data drift:", p)

# STEP 31: Handle High-Cardinality Categorical Data
# Use the flattened version from Details
df['Details_City'] = df['Details_City'].apply(
    lambda x: x if df['Details_City'].value_counts()[x] > 1 else 'Other'
)

# STEP 32: Handle Time-Based Data (Resample)
df.set_index('Date', inplace=True)
df_resampled = df.resample('D').sum(numeric_only=True)

# STEP 33: Detect Anomalies using Z-score
df['Z_score'] = zscore(df['Values'].fillna(0))
df['Anomaly'] = df['Z_score'].abs() > 2

# STEP 34: Clean Text Data
df['Text'] = df['Text'].astype(str)
df['Cleaned_Text'] = df['Text'].str.translate(str.maketrans('', '', string.punctuation)).str.lower()

# STEP 35: Geospatial Data (Simulated)
df = df[(df['Latitude'].between(-90, 90)) & (df['Longitude'].between(-180, 180))] if 'Latitude' in df else df

# STEP 36: Handle Sparse Data
df = df.loc[:, (df != 0).any(axis=0)]



# STEP 38: Fix Encoding Issues
df['Text'] = df['Text'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# STEP 39: Inconsistent Units — convert to kg
df['Weight'] = df['Weight'].apply(lambda x: float(x.replace('kg','')) if 'kg' in x else float(x.replace('g','')) / 1000)

print("✅ All 39 cleaning steps completed!")
df.head()


Name           0
Age            0
Gender         0
Date           0
Sales_Q1       0
Sales_Q2       0
Weight         0
Text           0
Values         0
Feature1       0
Feature2       0
Feature3       0
Sparse1        0
Sparse2        0
Class          0
Details        0
Future_Info    0
dtype: int64
Negative Ages: Empty DataFrame
Columns: [Name, Age, Date, Sales_Q1, Sales_Q2, Weight, Text, Values, Feature1, Feature2, Feature3, Sparse1, Sparse2, Class, Details, Future_Info, FirstName, LastName, FullDate, Gender_male]
Index: []
Skewness of Values: 2.4486297379727446
P-value for data drift: 0.4444444444444445
✅ All 39 cleaning steps completed!


<ipython-input-15-d8dc8609e737>:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
<ipython-input-15-d8dc8609e737>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Weight'] = df['Weight'].apply(lambda x: float(x.replace('kg','')) if 'kg' in x else float(x.replace('g','')) / 1000)


,Name,Age,Sales_Q1,Weight,Text,Values,Feature1,Class,FirstName,LastName,FullDate,Gender_male,Smoothed,Details_Age,Details_City,Z_score,Cleaned_Text
Date,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00+00:00,Alice,0.946619,100,10.0,Hello!,2.397895,1,A,Alice,,2023-01-01,True,NaN,25,Chennai,-0.812999,hello
2023-01-02 00:00:00+00:00,Bob,0.653025,200,15.0,Data@Science,2.564949,2,A,Bob,,2023-01-02,True,2.481422,30,Mumbai,-0.747382,datascience
2023-01-03 00:00:00+00:00,Charly,1.000000,300,20.0,Python#3,2.772589,3,A,Charly,,2023-01-03,False,2.668769,35,Other,-0.665824,python3
2023-01-01 00:00:00+00:00,Alice,0.946619,100,10.0,ANOTHER!,0.000000,1,A,Alice,,2023-01-01,True,1.386294,25,Chennai,-1.754864,another
2023-01-01 00:00:00+00:00,Bob,0.973310,200,10.0,Hello!,2.397895,2,A,Bob,,2023-01-01,True,1.198948,30,Mumbai,-0.812999,hello
